In [299]:
import cv2
import matplotlib.pyplot as plt
import numpy as np 
import os
from numpy import linalg
import itertools

In [307]:
def DLT(corners,world_pt):
    M = []
    for i in range(20):
        X,Y,Z = world_pt[i,0],world_pt[i,1],world_pt[i,2]
        u,v = corners[i,0],corners[i,1]
        a_x = [-X,-Y,-Z,-1,0,0,0,0,u*X,u*Y,u*Z,u]
        a_y = [0,0,0,0,-X,-Y,-Z,-1,v*X,v*Y,v*Z,v]
        M.append(a_x)
        M.append(a_y)
    M = np.array(M)
    M = M.reshape(2*20,12)
    
    U,S,Vh = np.linalg.svd(M)
    L = Vh[-1,:] /Vh[-1,-1]
    
    #Camera projection matrix:
    P = L.reshape(3,4)
    print("Projection Matrix")
    print(P)
    
    
    
    H = P[0:3,0:3]
    h = P[:,-1]

    
    
    H_inv = np.linalg.pinv(H)
    #projection center X0= -H_inv.h
    Proj_center = -1*np.matmul(H_inv,h)
    print("projection_center")
    print(Proj_center)
    
    #QR Decomposition of H_inv for getting rotation and calibration matrix 
    q, r = np.linalg.qr(H_inv)
    
    
    
    K = np.linalg.pinv(r)
    K = r/r[-1,-1]
    print("Camera Calibration matrix")
    print(K)
    
    print("Rotation matrix")
    Rot_mat = q.transpose()
    print(Rot_mat)
    
#     PX = np.matmul(P,world_pt[pt,:].reshape(4,1))
    error = 0
    for pt in range(20):
        PX = np.matmul(P,world_pt[pt,:].reshape(4,1))
        PX = PX/PX[-1,-1]
        error += (np.linalg.norm((corners[pt]-PX)))

    error = error/20
    print("DLT Error")
    print(error)
    return K

In [301]:
def DLT_reproj(world_pt,corners):
    pts = range(20)
    comb = list(itertools.combinations(pts, 6))
    no = len(comb)
    flag = 0
        
    for j in range(no):
        M_estimate = []

        for i in comb[j]:     
            X,Y,Z = world_pt[i,0],world_pt[i,1],world_pt[i,2]
            u,v = corners[i,0],corners[i,1]
            a_x = [-X,-Y,-Z,-1,0,0,0,0,u*X,u*Y,u*Z,u]
            a_y = [0,0,0,0,-X,-Y,-Z,-1,v*X,v*Y,v*Z,v]
            M_estimate.append(a_x)
            M_estimate.append(a_y)    
            
        M_estimate = np.array(M_estimate)
        M_estimate = M_estimate.reshape(2*6,12)

        U,S,Vh = np.linalg.svd(M_estimate)
        L = Vh[-1,:]/Vh[-1,-1]

        #Camera projection matrix:
        P_found = L.reshape(3,4)
        
        error=0
        
        for pt in range(20):
            PX = np.matmul(P_found,world_pt[pt,:].reshape(4,1))
            PX = PX/PX[-1,-1]
            error += (np.linalg.norm((corners[pt]-PX)))
        
        error = error/20
        
        if flag==0:
            least_error = error
            flag = 1
            P_opt = P_found
        
        if error < least_error:
            P_opt = P_found
            least_error = error
    
    print("Best P")
    print(P_opt)
    print("Least Error")
    print(least_error)
        
    return P_opt    


In [302]:

%matplotlib qt 
# corners = np.array([[630.499,1903.5,1],[1153.5,2689.5,1],[1280.5,1977.5,1],[708.504,3211.49,1],[2055.5,2408.5,1],[1657.5,2903.49,1],
#            [1658.5,2441.49,1],[2190.5,2989.5,1],[1904.5,3022.5,1],[2047.5,2755.5,1],[1289.51,2640.49,1],[1907.49,2700.49,1],
#            [1788.52,1981.46,1],[688.504,2861.49,1],[1166.5,3015.5,1],[1417.51,2279.49,1],[840.528,2444.47,1],[1285.54,2316.52,1]
#            ,[1777.5,2960.5,1],[1001.5,2572.49,1]])
# world_pt = np.array([[0,1,6,1],[0,6,3,1],[0,2,2,1],[0,8,6,1],[4,4,0,1],[1,8,0,1],
#             [1,5,0,1],[5,7,0,1],[3,8,0,1],[4,6,0,1],[0,6,2,1],[3,6,0,1],
#             [2,2,0,1],[0,6,6,1],[0,8,3,1],[0,4,1,1],[0,4,5,1],[0,4,2,1],
#             [2,8,0,1],[0,5,4,1]]).astype(np.float32)

world_pt=np.matrix([[0,28,28*6,1],[0,28,28*3,1],[0,28,0,1],[28*3,28,0,1],[28*6,28,3,1],
                      [0,28*3,28*6,1],[0,28*3,28*3,1],[0,28*3,0,1],[28*3,28*3,0,1],[28*6,28*3,3,1],
                      [0,28*5,28*6,1],[0,28*5,28*3,1],[0,28*5,0,1],[28*3,28*5,0,1],[28*6,28*5,3,1],
                      [0,28*7,28*6,1],[0,28*7,28*3,1],[0,28*7,0,1],[28*3,28*7,0,1],[28*6,28*7,3,1]])

corners = np.matrix([[627, 1908,1], [1136, 1829,1], [1549, 1764,1], [1926, 1829,1], [2413, 1922,1], [658, 2299,1], 
            [1145, 2185,1], [1549, 2093,1], [1922, 2189,1], [2387, 2321,1], [688, 2685,1], [1145, 2523,1], 
            [1540, 2400,1], [1913, 2540,1], [2374, 2698,1], [706, 3032,1], [1158, 2856,1], [1544, 2707,1], 
            [1913, 2861,1], [2352, 3062,1]])

# world_pt = world_pt*28
# o= np.ones(20)
# print(o.shape())
# np.append(world_pt,,axis=1)
print(world_pt.shape)

image = cv2.imread('Camera_calibration_data/calib-object.jpg')
# plt.figure()
# plt.imshow(image)
print(world_pt.shape)


(20, 4)
(20, 4)


In [308]:
# corners = corners.reshape(-1,3)
print (corners.shape)
# print (corners)
# print (world_pt.shape)
# print(world_pt)

K = DLT(corners,world_pt)


(20, 3)
Projection Matrix
[[ 2.43230665e+00  6.06749709e-01 -6.16125164e+00  1.54083681e+03]
 [-1.36811875e+00  6.68999676e+00 -1.24293395e+00  1.59596820e+03]
 [-1.18597072e-03  3.82948419e-04 -1.10526986e-03  1.00000000e+00]]
projection_center
[433.70355384 -72.92804573 414.11841734]
Camera Calibration matrix
[[ 2.70167789e-04 -4.48008173e-07 -4.03902385e-01]
 [-0.00000000e+00  2.73707613e-04 -5.48268951e-01]
 [-0.00000000e+00 -0.00000000e+00  1.00000000e+00]]
Rotation matrix
[[-0.68242233 -0.00375398  0.73094848]
 [-0.16554867 -0.97320881 -0.15955641]
 [ 0.71196447 -0.22989241  0.66351795]]
DLT Error
4371.0482509086305


In [304]:
DLT_reproj(world_pt,corners)

/Users/GowriL/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: divide by zero encountered in true_divide
/Users/GowriL/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in true_divide
/Users/GowriL/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in matmul
/Users/GowriL/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in true_divide
/Users/GowriL/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in true_divide


Best P
[[-1.84404762e+01  1.68719534e-11 -9.22023810e+00  1.54900000e+03]
 [-2.10000000e+01 -7.49692470e-11 -1.05000000e+01  1.76400000e+03]
 [-1.19047619e-02 -7.28219745e-14 -5.95238095e-03  1.00000000e+00]]
Least Error
4004.7804011477374


# Correcting Radial Distortion 

In [333]:
distorted_imag = cv2.imread('Camera_calibration_data/Fig1.png')
# distorted_imag_gry = cv2.cvtColor(distorted_imag, cv2.COLOR_RGB2GRAY)

dist_im_pts = np.array([[1595,37],[504,196],[1169,141],[1758,262],[655,336],[1290,344],[796,475],[1428,563],[366,767],[924,599],[1567,778],[71,1590],[659,1343],[1039,1222],[1550,1486],[220,1365],[792,1118],[1413,1228],[198,1002],[797,799],[1154,810],[1715,1008]]).astype(np.float32)
dist_w_pts = np.array([[126,14,0],[0,42,98],[42,42,0],[154,42,0],[0,70,70],[70,70,0],[0,98,42],[98,98,0],[0,126,126],[0,126,14],[126,126,0],[0,238,182],[0,238,70],[14,238,0],[126,238,0],[0,210,154],[0,210,42],[98,210,0],[0,154,154],[0,154,42],[42,154,0],[154,154,0]]).astype(np.float32)
camera_matrix = K
camera_matrix[0,1]=0
camera_matrix[0,2]=0
camera_matrix[1,2]=0

dist_im_pts = np.array([[1595,37],[504,196],[1169,141],[1758,262],[655,336],[1290,344],[796,475],[1428,563],[366,767],[924,599],[1567,778],[71,1590],[659,1343],[1039,1222],[1550,1486],[220,1365],[792,1118],[1413,1228],[198,1002],[797,799],[1154,810],[1715,1008]]).astype(np.float32)
dist_w_pts = np.array([[126,14,0],[0,42,98],[42,42,0],[154,42,0],[0,70,70],[70,70,0],[0,98,42],[98,98,0],[0,126,126],[0,126,14],[126,126,0],[0,238,182],[0,238,70],[14,238,0],[126,238,0],[0,210,154],[0,210,42],[98,210,0],[0,154,154],[0,154,42],[42,154,0],[154,154,0]]).astype(np.float32)

# ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(dist_w_pts,dist_im_pts, (distorted_imag.shape[1],distorted_imag.shape[0]), None, None)

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(dist_w_pts, dist_im_pts, distorted_imag_gry.shape[::-1],None,None)

h,  w = distorted_imag_gry.shape[:2]
newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))

dst = cv2.undistort(distorted_imag_gry, mtx, dist, None, newcameramtx)

# crop the image
x,y,w,h = roi
dst = dst[y:y+h, x:x+w]
plt.figure()
plt.imshow(dst)




undistorted_img = cv2.undistort(distorted_imag, mtx, dist, None, newcameramtx)

cv2.imwrite('calib-undistort.png', undistorted_img)



New Camera Matrix
[[3.73681571e-04 0.00000000e+00 7.25075131e-39]
 [0.00000000e+00 2.90346186e-04 7.66386162e-39]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
(0, 0, 1016, 501)


True